In [ ]:
from ecuaciones import Ecuaciones 
from methods import Methods as m
from experiment import Experiment as ex

import math
import pandas
import pandas as pd
import os

In [ ]:
import experiment
experiment.Experiment.experiment(10, 1000)

In [ ]:
import numpy as np
def optimize_hyperparameters(fitness_function, num_genes, num_iterations, maximize=False, metodo = None, repetitions = 30):

    best_median = float('inf')
    best_params = None
    best_solution = None
    # Definición de espacios de búsqueda para cada hiperparámetro
    num_individuals_list = [50, 100, 150, 200,300,400,500]
    percentage_best_list = [0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1]
    mutation_prob_list = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4,0.5]

    
    # Recorrer cada combinación de parámetros
    for num_individuals in num_individuals_list:
        for percentage_best in percentage_best_list:
            for mutation_prob in mutation_prob_list:
                fitness_values = []
                solutions = []
                for rep in range(repetitions):
                    solution, convergence = metodo(
                        fitness_function=fitness_function,
                        num_individuals=num_individuals,
                        num_genes=num_genes,
                        percentage_best=percentage_best,
                        mutation_prob=mutation_prob,
                        num_iterations=num_iterations,
                        maximize=maximize
                    )
                    fitness = fitness_function(solution)
                    fitness_values.append(fitness)
                    solutions.append(solution)
                    
                median_fitness = np.median(fitness_values)
                print(f'num_individuals: {num_individuals}, percentage_best: {percentage_best}, mutation_prob: {mutation_prob}, median fitness: {median_fitness}')
                
                # Actualizar la mejor configuración si la mediana es menor
                if median_fitness < best_median:
                    best_median = median_fitness
                    best_params = (num_individuals, percentage_best, mutation_prob)
                    # Se selecciona la mejor solución de las repeticiones actuales
                    best_solution = solutions[np.argmin(fitness_values)]

    print(f'\nMejores parámetros encontrados: num_individuals = {best_params[0]}, '
          f'percentage_best = {best_params[1]}, mutation_prob = {best_params[2]} con median fitness = {best_median}')
    
    return best_solution, best_params, best_median



In [ ]:
xd = m(-10,10)
funciones = [Ecuaciones.F3, Ecuaciones.F4, Ecuaciones.F5, Ecuaciones.F11]
metodo = xd.evolutionary_algorithm
configuraciones = []


In [ ]:
from multiprocessing import Pool

def optimize_single_function(args):
    func, metodo, num_genes, num_iterations, maximize = args
    best_solution, best_params, best_fitness = optimize_hyperparameters(
        fitness_function=func,
        num_genes=num_genes,
        num_iterations=num_iterations,
        maximize=maximize,
        metodo=metodo
    )
    return best_params

def optimize_configurations_parallel(funciones, metodo, num_genes=5, num_iterations=1000, maximize=False):
    args = [(func, metodo, num_genes, num_iterations, maximize) for func in funciones]
    with Pool() as pool:  # Usa todos los núcleos disponibles
        configuraciones = pool.map(optimize_single_function, args)
    return configuraciones

# Uso:
configuraciones = optimize_configurations_parallel(funciones, metodo)

In [ ]:
configuraciones